In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Fungsi untuk memformat nominal uang
def format_currency(x, pos):
    if x >= 1e9:
        return f'{x/1e9:.1f}B'
    elif x >= 1e6:
        return f'{x/1e6:.1f}M'
    elif x >= 1e3:
        return f'{x/1e3:.1f}K'
    else:
        return f'{x:.0f}'

# Mengambil data dari API
response = requests.get("http://localhost:3000/api/sales")
data = pd.read_json(response.text)

# Mengubah kolom 'date' menjadi tipe datetime
data['date'] = pd.to_datetime(data['date'])

# Menghitung pendapatan (price * quantity)
data['revenue'] = data['price'] * data['quantity']

# Analisa pendapatan harian dalam rentang waktu bulan (menggunakan bulan Januari 2024 sebagai contoh)
daily_revenue = data[(data['date'].dt.year == 2024) & (data['date'].dt.month == 1)].groupby(data['date'].dt.date)['revenue'].sum().reset_index()

plt.figure(figsize=(12, 6))
bars = plt.bar(daily_revenue['date'], daily_revenue['revenue'])
plt.xlabel('Tanggal')
plt.ylabel('Pendapatan')
plt.title('Pendapatan Harian - Januari 2024')
plt.xticks(rotation=45)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(format_currency))

# Menambahkan nominal pada atas chart
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{format_currency(height, 0)}',
             ha='center', va='bottom', rotation=0)

plt.tight_layout()
plt.show()

# Analisa pendapatan bulanan dalam rentang waktu tahunan (2024)
monthly_revenue = data[data['date'].dt.year == 2024].groupby(data['date'].dt.to_period('M'))['revenue'].sum().reset_index()
monthly_revenue['date'] = monthly_revenue['date'].dt.to_timestamp()

plt.figure(figsize=(12, 6))
bars = plt.bar(monthly_revenue['date'], monthly_revenue['revenue'])
plt.xlabel('Bulan')
plt.ylabel('Pendapatan')
plt.title('Pendapatan Bulanan - 2024')
plt.xticks(monthly_revenue['date'], [date.strftime('%B') for date in monthly_revenue['date']], rotation=45)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(format_currency))

# Menambahkan nominal pada atas chart
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{format_currency(height, 0)}',
             ha='center', va='bottom', rotation=0)

plt.tight_layout()
plt.show()